In [ ]:
#| label: t2Fig2jn

# Prepare Python environment

import scipy.io as sio
from pathlib import Path

from contextlib import contextmanager
import sys, os
from pathlib import Path

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

import os
from pathlib import Path

def find_myst_yml_directories(start_dir=None):
    """
    Recursively search for directories containing myst.yml file.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        list: List of full paths to directories containing myst.yml
    """
    if start_dir is None:
        start_dir = Path.cwd()
    else:
        start_dir = Path(start_dir)
    
    myst_dirs = []
    
    def _search_directory(current_dir):
        # Check if myst.yml exists in current directory
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            myst_dirs.append(str(current_dir.resolve()))
            # Don't search subdirectories if we found myst.yml here
            return
        
        # Recursively search all subdirectories
        for item in current_dir.iterdir():
            if item.is_dir():
                try:
                    _search_directory(item)
                except (PermissionError, OSError):
                    # Skip directories we can't access
                    continue
    
    _search_directory(start_dir)
    return myst_dirs

def find_myst_yml_directories_upwards(start_dir=None):
    """
    Search for myst.yml in current directory, if not found go to parent and repeat.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        str or None: Full path of directory containing myst.yml, or None if not found
    """
    if start_dir is None:
        current_dir = Path.cwd()
    else:
        current_dir = Path(start_dir)
    
    # Keep going up until we reach the filesystem root
    while current_dir != current_dir.parent:  # Stop at root
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            return str(current_dir.resolve())
        
        # Move to parent directory
        current_dir = current_dir.parent
    
    return None

with suppress_stdout():
    repo_path = Path(find_myst_yml_directories_upwards())
    print(repo_path)
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
data_dir = dataset_path / "03-T2" / "03-T2"  
data_file = "t2_and_t2star_curvs.mat"

#Load either archived or generated plot variables
mat_contents = sio.loadmat(data_dir / data_file)

# Get the signals and parameters from Matlab

# T2
T2_signal_WM = mat_contents['signal_WM_T2'][0]
T2_signal_GM = mat_contents['signal_GM_T2'][0]

# T2*
T2star_signal_WM = mat_contents['signal_WM_T2star'][0]
T2star_signal_GM = mat_contents['signal_GM_T2star'][0]

# TE 
params = mat_contents['params']
TE = mat_contents['params']['TE'][0][0][0]

## Plot

import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly.offline import iplot

config={'showLink': False, 'displayModeBar': False}

# T2 signals

wm_T2 = go.Scatter(
    x = TE,
    y = T2_signal_WM,
    name = 'T<sub>2</sub> = 109.77 ms (White Matter)',
    text = 'T<sub>2</sub> = 109.77 ms (White Matter)',
    hoverinfo = 'x+y+text',
    line=dict(color='#1f77b4', dash='solid'),
    visible = True
)

gm_T2 = go.Scatter(
    x = TE,
    y = T2_signal_GM,
    name = 'T<sub>2</sub> = 96.07 ms (Gray Matter)',
    text = 'T<sub>2</sub> = 96.07 ms (Gray Matter)',
    hoverinfo = 'x+y+text',
    line=dict(color='#ff7f0e', dash='solid'),
    visible = True
)

# T2* signals

wm_T2star = go.Scatter(
    x = TE,
    y = T2star_signal_WM,
    name = 'T<sub>2</sub>* = 67.63 ms (White Matter)',
    text = 'T<sub>2</sub>* = 67.63 ms (White Matter)',
    hoverinfo = 'x+y+text',
    line=dict(color='#1f77b4', dash='dot'),
    visible = False
)

gm_T2star = go.Scatter(
    x = TE,
    y = T2star_signal_GM,
    name = 'T<sub>2</sub>* = 48.48 ms (Gray Matter)',
    text = 'T<sub>2</sub>* = 48.48 ms (Gray Matter)',
    hoverinfo = 'x+y+text',
    line=dict(color='#ff7f0e', dash='dot'),
    visible = False
)

data = [wm_T2, gm_T2, wm_T2star, gm_T2star]

layout = go.Layout(
    width=600,
    height=375,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=20,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.175,
            showarrow=False,
            text='Echo Time – TE (ms)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.50,
            showarrow=False,
            text='Transverse Magnetization (M<sub>xy</sub>)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2,
        range=[0, 1]
    ),
    legend=dict(
        x=0.53,
        y=0.97,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ),
    updatemenus=[
        dict(
            buttons=list([
                # Display T2
                dict(
                    args=[{'visible': [True, True, False, False]}],
                    label='T2',
                    method='update'
                ),
                # Display T2*
                dict(
                    args=[{'visible': [False, False, True, True]}],
                    label='T2*',
                    method='update'
                ),
                # Display both T2 and T2*
                dict(
                    args=[{'visible': [True, True, True, True]}],
                    label='T2 and T2*',
                    method='update'
                ),
            ]),
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=0.30,
            xanchor='left',
            y=1.0,
            yanchor='top',
            font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
            )
        ),
    ]
) 

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'ir_fig_2.html', config = config)